In [11]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
import nltk
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model

In [9]:
data = pd.read_csv('/content/IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
english_stops = set(stopwords.words('english'))

In [15]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


/tmp/ipython-input-4067306172.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [16]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
9097     [this, one, greatest, films, ever, made, brill...
24957    [this, film, plunges, headlong, realm, surreal...
20862    [alfred, hitchcok, favorite, director, means, ...
7640     [this, documentary, first, introduction, peak,...
13443    [such, pain, pain, shape, promise, central, id...
                               ...                        
21150    [this, movie, starts, brisk, slow, moments, mi...
17240    [i, got, much, testosterone, next, bloke, raqu...
19105    [this, movie, funny, amitabh, bachan, govinda,...
3930     [don, waste, time, one, cool, looking, boxes, ...
7361     [yes, plot, predictable, yes, plot, holes, yes...
Name: review, Length: 40000, dtype: object 

2921     [simply, imaginatively, filmed, studio, set, p...
31593    [allegedly, true, story, juana, de, castilla, ...
30765    [this, movie, could, lot, potential, certainly...
47879    [william, powell, final, outing, philo, vance,...
74       [upon, viewing, tobe, hooper, gem, crocodile, ...
 

In [17]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [18]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    8     5   697 ...     0     0     0]
 [    8     4 16610 ...     0     0     0]
 [ 3980 39937   434 ...     0     0     0]
 ...
 [    8     3    67 ...     0     0     0]
 [  336   356    10 ...   181    49  3175]
 [  319    40   620 ...     0     0     0]] 

Encoded X Test
 [[  238 22696   638 ...     0     0     0]
 [ 7808   192    15 ...   219    14   433]
 [    8     3    26 ...     0     0     0]
 ...
 [   23  2785     4 ...     1  1632    51]
 [    1    13    82 ...     0     0     0]
 [   96   286 21327 ... 87677  2803  2136]] 

Maximum review length:  130


In [19]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [20]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [21]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - accuracy: 0.6200 - loss: 0.6079
Epoch 1: accuracy improved from -inf to 0.74107, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 78s 241ms/step - accuracy: 0.6203 - loss: 0.6076
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.8260 - loss: 0.4385
Epoch 2: accuracy improved from 0.74107 to 0.78385, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 77s 227ms/step - accuracy: 0.8259 - loss: 0.4388
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.6628 - loss: 0.6145
Epoch 3: accuracy did not improve from 0.78385
313/313 ━━━━━━━━━━━━━━━━━━━━ 81s 223ms/step - accuracy: 0.6625 - loss: 0.6146
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - accuracy: 0.5783 - loss: 0.6549
Epoch 4: accuracy did not improve from 0.78385
313/313 ━━━━━━━━━━━━━━━━━━━━ 79s 215ms/step - accuracy: 0.5785 - loss: 0.6548
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.7648 - loss: 0.4938
Epoch 5: accuracy did not improve from 0.78385
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 216ms/step - accuracy: 0.7646 - loss: 0.4940


In [23]:
y_pred_prob = model.predict(x_test, batch_size = 128)
y_pred = (y_pred_prob > 0.5).astype(int)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

79/79 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step
Correct Prediction: 6853
Wrong Prediction: 3147
Accuracy: 68.53


In [24]:
loaded_model = load_model('models/LSTM.h5')

In [25]:
review = str(input('Movie Review: '))

Movie Review: 5


In [26]:
# Pre-process input
import re
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  
Filtered:  ['']


In [27]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [28]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
[[0.7068048]]


In [29]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
